In [1]:
# from mpi4py import MPI

import os
import subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time as t
# import io

# import datetime

import astropy.units as u
import astropy.constants as c

# from IPython.display import Image, display_png


In [2]:
%matplotlib inline
# %matplotlib notebook

plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["mathtext.fontset"] = "stix"
plt.rcParams["font.size"] = 15


In [3]:
filepath_ = "../../hewd/gamtmp.d"
# f_ = open(filepath_, mode="r") 

In [4]:
filepath_[:-len(filepath_.split("/")[-1])]

'../../hewd/'

In [15]:
def read_gamtmp_save_npy(filepath, force=False):
    """
    read gamtmp.d, save the data to npy files, and return the data as numpy.ndarray
    
    arguments
    =========
    filepath: string        : filepath of gamtmp.d
    force   : bool(optional): default=False, force to read gamtmp.d even if npy files already exist
    
    return
    ======
    nobj      : int                 : number of wavelength bin
    wava      : float, numpy ndarray: (mean) wavelength array
    head_table: float, numpy ndarray: header tables: time, theta, phi
    data_table: float, numpy ndarray: data tables (nobj)
    """
    
    directory = filepath[:-len(filepath.split("/")[-1])]
    with open(filepath) as f:
        lines = f.readlines()
    nobj = np.int(lines[0]) - 1
    wava = np.array(lines[1].split("  ")[1:], dtype=np.float)
    
    if(os.path.exists(directory+"header.npy") and not force):
        head_table = np.load(directory+"header.npy")
    else:
        head_ = lines[2::2]
        with open(directory+"header.txt", mode="w") as f:
            f.writelines(head_)
        head_table = np.loadtxt(directory+"header.txt", dtype=np.float)
        np.save(directory+"header.npy", head_table)
        os.remove(directory+"header.txt")
    
    if(os.path.exists(directory+"data.npy") and not force):
        data_table = np.load(directory+"data.npy")
    else:
        data_ = lines[3::2]
        with open(directory+"data.txt", mode="w") as f:
            f.writelines(data_)
        data_table = pd.read_csv(directory+"data.txt", delim_whitespace=True, header=None, dtype=np.float)
        np.save(directory+"data.npy", data_table)
        os.remove(directory+"data.txt")

    return nobj, wava, head_table, data_table

def extract_array_from_header(head_table: np.ndarray):
    time_arr  = np.unique(head_table[:, 0])
    theta_arr = np.sort(np.unique(head_table[:, 1]))
    phi_arr   = np.sort(np.unique(head_table[:, 2]))
    return time_arr, theta_arr, phi_arr

# def read_npy(directory):

In [16]:
%time __, wava, head_table, data_table = read_gamtmp_save_npy(filepath_)
time_arr, theta_arr, phi_arr = extract_header(head_table)

CPU times: user 5.58 s, sys: 10.3 s, total: 15.8 s
Wall time: 18.1 s


In [ ]:
def set_parameters():
      parameter(fvega_ju=4.2631d-9) #erg/s/cm2/A
      parameter(fvega_jb=6.1855d-9)
      parameter(fvega_jv=3.6021d-9)
      parameter(fvega_jr=2.1422d-9)
      parameter(fvega_ji=1.1117d-9)
      parameter(fvega_jj=3.0949d-10)
      parameter(fvega_jh=1.1534d-10)
      parameter(fvega_jk=4.4936d-11)
      parameter(fvega_fu=3.91476193d-9)
      parameter(fvega_fb=6.18069247d-9)
      parameter(fvega_fv=3.64177726d-9)
      parameter(fvega_fr=2.09980855d-9)
      parameter(fvega_fi=1.12042287d-9)